# Behavior Cloning

### Data Augmentation and Formatting
Flip images and measurements

In [14]:
import csv
import cv2
import numpy as np

In [15]:
lines = []
with open("./data/driving_log.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0].split('/')
        if len(filename) > 3:
            filename = '/'.join(filename[-2],filename[-1]])
        lines.append([filename, *line[-4:]])

fw = open("./data/driving_log_updated.csv", "w")
for line in lines:
    img = cv2.imread(line[0], cv2.IMREAD_COLOR)
    img = np.fliplr(img)
    new_filename = line[0][:-4]+"_flipped.jpg"
    fw.write("{},{},{},{},{}\n".format(*line))
    fw.write("{},{},{},{},{}\n".format(new_filename, -1.0 * float(line[1]), -1.0 * float(line[2]), -1.0 * float(line[3]), -1.0 * float(line[4])))
    cv2.imwrite(new_filename, img)
    
fw.close()

IndexError: list index out of range

Crop Images

In [10]:
# 55 off top
# 30 off bottom
fw = open("./data/driving_log_updated_cropped.csv", "w")
with open("./data/driving_log_updated.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        filename = line[0]
        img = cv2.imread(filename)
        img = img[55:-30, :]
        f_split = filename.split("/")
        f_split[-2] = "CROPPED_IMG"
        new_filename = '/'.join(f_split)
        fw.write("{},{},{},{},{}\n".format(new_filename, *line[1:]))
        cv2.imwrite(new_filename, img)
    
fw.close()

### Define Datasets

In [11]:
import random
import csv

TEST_SPLIT_RATIO = 0.2
VALIDATION_SPLIT_RATIO = 0.1

train_data = []
test_data = []
validation_data = []

with open("./data/driving_log_updated_cropped.csv") as f:
    reader = csv.reader(f)
    for line in reader:
        rand = random.random()
        if rand < TEST_SPLIT_RATIO:
            test_data.append(line)
        elif rand < VALIDATION_SPLIT_RATIO+TEST_SPLIT_RATIO:
            validation_data.append(line)
        else:
            train_data.append(line)


In [13]:
total = 1.0* len(train_data)+len(test_data)+len(validation_data)
print(len(train_data), len(train_data)/total )
print(len(test_data), len(test_data)/total)
print(len(validation_data), len(validation_data)/total)


9581 0.6963948248291902
2835 0.20606192760575665
1342 0.09754324756505306


In [4]:
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size-1]

            vals = np.zeros((batch_size, 4))
            imgs = np.zeros((batch_size, 75, 320, 3))
            index = 0
            for batch_sample in batch_samples:
                filename = batch_sample[0]
                center_image = cv2.imread(filename)
                imgs[index] = center_image
                index += 1
                try:
                    float(batch_sample[1])
                    float(batch_sample[2])
                    float(batch_sample[3])
                    float(batch_sample[4])
                except:
                    print(batch_sample[1], batch_sample[2],batch_sample[3],batch_sample[4])
                vals[index][0] = batch_sample[1]
                vals[index][1] = batch_sample[2]
                vals[index][2] = batch_sample[3]
                vals[index][3] = batch_sample[4]

            # trim image to only see section with road
            X_train = imgs
            y_train = vals
            yield X_train, y_train


### Define Hyperparameters

In [5]:
epochs = 50
batch_size = 128

train_data_size = len(train_data)
test_data_size = len(test_data)
validation_data_size = len(validation_data)

### Define Model and Train

In [6]:
train_generator = generator(train_data, batch_size=batch_size)
validation_generator = generator(validation_data, batch_size=batch_size)
test_generator = generator(test_data, batch_size=batch_size)

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Lambda, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.advanced_activations import ELU
from keras.optimizers import Adam

model = Sequential()
model.add(Lambda(lambda x: x/127.5 - 1.0, input_shape=(75, 320, 3)))
model.add(Conv2D(16, (8,8), strides=(4, 4)))
model.add(ELU())
model.add(Conv2D(32, (5,5), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Conv2D(64, (3,3), strides=(2, 2)))
model.add(ELU())
model.add(Flatten())
model.add(Dropout(.2))
model.add(Dense(1024))
model.add(ELU())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(ELU())
model.add(Dense(64))
model.add(ELU())
model.add(Dense(4))


Using TensorFlow backend.


In [8]:
model.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['accuracy'])

In [9]:
from keras.callbacks import ModelCheckpoint, Callback
checkpoint_callback = ModelCheckpoint('model{epoch:02d}.h5')
model.fit_generator(train_generator, steps_per_epoch=train_data_size/batch_size, \
                   validation_data=validation_generator, validation_steps=validation_data_size/batch_size, \
                   epochs=50, max_q_size=1, callbacks=[checkpoint_callback])

Epoch 1/50
76/75 [==============================] - 19s - loss: 145.1195 - acc: 0.4258 - val_loss: 153.8184 - val_acc: 0.3561
Epoch 2/50
76/75 [==============================] - 13s - loss: 132.3306 - acc: 0.4626 - val_loss: 162.7625 - val_acc: 0.3672
Epoch 3/50
76/75 [==============================] - 13s - loss: 125.7198 - acc: 0.4855 - val_loss: 166.7925 - val_acc: 0.3457
Epoch 4/50
76/75 [==============================] - 13s - loss: 123.4035 - acc: 0.4998 - val_loss: 170.4733 - val_acc: 0.3509
Epoch 5/50
76/75 [==============================] - 13s - loss: 121.9788 - acc: 0.5032 - val_loss: 172.6738 - val_acc: 0.3542
Epoch 6/50
76/75 [==============================] - 13s - loss: 120.7160 - acc: 0.5013 - val_loss: 175.9094 - val_acc: 0.3424
Epoch 7/50
76/75 [==============================] - 13s - loss: 119.6952 - acc: 0.5093 - val_loss: 178.3333 - val_acc: 0.3490
Epoch 8/50
76/75 [==============================] - 13s - loss: 118.6419 - acc: 0.5098 - val_loss: 180.7816 - val_acc:

KeyboardInterrupt: 

### Evaluate

In [ ]:
metrics = evaluate_generator(test_generator, steps=test_data_size/batch_size)

In [ ]:
print(zip(model.metrics_names, metrics))